# Define Model

In [1]:
from utils.DevConf import DevConf
DEV_CONF = DevConf(device='cuda')

NameError: name '_C' is not defined

In [ ]:
from utils.AttnBlocksConf import AttnBlocksConf
from model.BertDecoder.SentiClassifier import SentiClassifier
from model.CombinationModel import CombinationModel
from utils.const import BlockType

d:\code\python\bert\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
mapper = SentiClassifier(6, AttnBlocksConf(768, 12, nKVHead=6), BlockType.CROSS)
model = CombinationModel(nClass=6, decoder=mapper, devConf=DEV_CONF)

d:\code\python\bert\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Load Data

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased", cache_dir='./cache/tokenizer')

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('data/archive/train.csv')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx]['ABSTRACT']
        label = torch.tensor([self.df.iloc[idx][i] for i in ["Computer Science","Physics","Mathematics","Statistics","Quantitative Biology","Quantitative Finance"]])
        return text, label
        # return self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128), label

In [ ]:
def collect_fn(batch):
    texts, labels = zip(*batch)
    # print(texts)
    return tokenizer(texts, return_tensors='pt', padding='max_length', truncation=True, max_length=512).to(device=DEV_CONF.device), torch.stack(labels).to(DEV_CONF.device)

In [ ]:
train_dataset = MyDataset(train, tokenizer)

datasize = len(train_dataset)
splitIndex = int(datasize * 0.2)

test_dataset = train_dataset[:splitIndex]
train_dataset = train_dataset[splitIndex:]

train_loader = DataLoader(train_dataset, collate_fn=collect_fn, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, collate_fn=collect_fn, batch_size=8, shuffle=True)

# Train

In [ ]:
from torch import nn

In [ ]:
lr = 1e-5
epochs = 1
loss_fn = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [ ]:
def train_fn(model, train_loader, loss_fn, optimizer, epochs):
    for epoch in range(epochs):
        for i, (data, label) in enumerate(train_loader):
            # print(data['input_ids'])
            # break
            optimizer.zero_grad()
            output = model(**data)
            loss = loss_fn(output, label.float())
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                print(f"Epoch {epoch+1}/{epochs} - Batch {i+1}/{len(train_loader)} - Loss: {loss.item()}")

In [ ]:
model.train()
train_fn(model, train_loader, loss_fn, optimizer, epochs)
print("Done")

Epoch 1/1 - Batch 1/2622 - Loss: 0.9170064330101013
Epoch 1/1 - Batch 101/2622 - Loss: 0.6943032741546631
Epoch 1/1 - Batch 201/2622 - Loss: 0.69356369972229
Epoch 1/1 - Batch 301/2622 - Loss: 0.6934887170791626
Epoch 1/1 - Batch 401/2622 - Loss: 0.6930962800979614
Epoch 1/1 - Batch 501/2622 - Loss: 0.6934904456138611
Epoch 1/1 - Batch 601/2622 - Loss: 0.6934870481491089
Epoch 1/1 - Batch 701/2622 - Loss: 0.6946806907653809
Epoch 1/1 - Batch 801/2622 - Loss: 0.6845504641532898
Epoch 1/1 - Batch 901/2622 - Loss: 0.6935328245162964
Epoch 1/1 - Batch 1001/2622 - Loss: 0.6776870489120483
Epoch 1/1 - Batch 1101/2622 - Loss: 0.6853147745132446
Epoch 1/1 - Batch 1201/2622 - Loss: 0.6559020280838013
Epoch 1/1 - Batch 1301/2622 - Loss: 0.6415303945541382
Epoch 1/1 - Batch 1401/2622 - Loss: 0.6483358144760132
Epoch 1/1 - Batch 1501/2622 - Loss: 0.671057403087616
Epoch 1/1 - Batch 1601/2622 - Loss: 0.6766590476036072
Epoch 1/1 - Batch 1701/2622 - Loss: 0.6568801999092102
Epoch 1/1 - Batch 1801/26

# Eval

In [ ]:
model.eval()

CombinationModel(
  (distilBert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1):

In [ ]:
id = 13
sample = tokenizer(train_dataset[id][0], return_tensors='pt', padding='max_length', truncation=True, max_length=512).to(device=DEV_CONF.device)
output = torch.where(model(**sample) > 0.2, 1, 0)
print(output)
print(train_dataset[id][1])
ans = torch.eq(output.to("cpu"), train_dataset[id][1])
print(ans)
print(torch.all(ans))

tensor([[0, 0, 1, 0, 0, 0]], device='cuda:0')
tensor([1, 0, 0, 0, 0, 0])
tensor([[False,  True, False,  True,  True,  True]])
tensor(False)


In [ ]:
# acc = [0] * 6
# testdata = train.sample(1000)
# test_dataset = MyDataset(testdata, tokenizer)

# for data in test_dataset:
#     # print(data[1][0])
#     sample = tokenizer(data[0], return_tensors='pt', padding='max_length', truncation=True, max_length=512).to(device=DEV_CONF.device)
#     output = torch.where(model(**sample) > 0.2, 1, 0)
#     ansList = torch.eq(output.squeeze().to("cpu"), data[1])
#     for i, ans in enumerate(ansList):
#         # print(ans)
#         if ans:
#             acc[i] += 1

In [ ]:
def test(model, test_loader):
    acc = [0] * 6
    for i, (data, label) in enumerate(test_loader):
        output = torch.where(model(**data) > 0.2, 1, 0)
        ansList = torch.eq(output.to("cpu"), label)
        for i, ans in enumerate(ansList):
            if ans:
                acc[i] += 1
    return acc

In [ ]:
acc = test(model, test_loader)

In [ ]:
print(acc)

[836, 919, 886, 770, 968, 992]
